In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from PIL import Image
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.utils import save_image


In [ ]:
model=models.vgg19(pretrained=True)

In [ ]:
model=model.features

0,5,10,19,28-> req. conv layers

In [ ]:
class VGG(nn.Module):
    def __init__(self):
        super(VGG, self).__init__()
        self.chosenlayers=[0,5,10,19,28]
        self.model=models.vgg19(pretrained=True).features[:29]

    def forward(self,x):
      ftrs=[]

      for num,layer in enumerate(self.model):
        x=layer(x) #Applying conv layer on x

        if num in self.chosenlayers:
          ftrs.append(x)
      return ftrs


def load_img(img_name):
  img=Image.open(img_name)
  img=loader(img).unsqueeze(0)

  return img.to(device)




In [ ]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
img_size=356

loader=transforms.Compose([
    transforms.Resize((img_size,img_size)),
    transforms.ToTensor(),
])


In [ ]:
import cv2
cap= cv2.VideoCapture('cont_vid.mp4')
i=0
print(1)
while 1:
  ret,frame=cap.read()

  if ret:
    cv2.imwrite(r"./data/cvid"+str(i)+".jpg",frame)
    i+=1
    print(i)
  else:
    print("done")
    break
cap.release()

In [ ]:
cimg=load_img("contn.jpg")
simg=load_img("style4.jpeg")

gen=cimg.clone().requires_grad_(True)

In [ ]:
model= VGG().to(device).eval() #eval is to freeze weights

In [ ]:
tot_steps=6000
lr=0.001
alpha=0.9
beta=0.01
optimizer=optim.Adam([gen],lr=lr)

Losses- content loss=mean((gen-cont)^2)
        style loss=

In [ ]:
for step in range(tot_steps):

  optimizer.zero_grad()
  gen_features=model(gen)
  content_features=model(cimg)
  style_features=model(simg)

  s_loss=c_loss=0

  for gf,cf,sf in zip(gen_features,content_features,style_features):
    b_size,c,h,w=gf.shape
    c_loss+=torch.mean((gf-cf)**2)

    G= lambda x : x.view(c,h*w).mm(x.view(c,h*w).t())
    gen_gram=G(gf)
    style_gram=G(sf)

    s_loss+=torch.mean((gen_gram- style_gram)**2)

  tot_loss=alpha*c_loss+beta*s_loss
  optimizer.zero_grad()
  tot_loss.backward()
  optimizer.step()

  if step %200 ==0:
    print(tot_loss)
    save_image(gen,'gen.png')


In [ ]:
#FOR VIDEO>:<:
tot_steps=1000
lr=0.001
alpha=0.9
beta=0.01

for i in range(90):
  cimg=load_img(r"./data/cvid"+str(i)+".jpg")
  simg=load_img("style4.jpeg")

  gen=cimg.clone().requires_grad_(True)

  model= VGG().to(device).eval() #eval is to freeze weights


  optimizer=optim.Adam([gen],lr=lr)

  for step in range(tot_steps):

    optimizer.zero_grad()
    gen_features=model(gen)
    content_features=model(cimg)
    style_features=model(simg)

    s_loss=c_loss=0

    for gf,cf,sf in zip(gen_features,content_features,style_features):
      b_size,c,h,w=gf.shape
      c_loss+=torch.mean((gf-cf)**2)

      G= lambda x : x.view(c,h*w).mm(x.view(c,h*w).t())
      gen_gram=G(gf)
      style_gram=G(sf)

      s_loss+=torch.mean((gen_gram- style_gram)**2)

    tot_loss=alpha*c_loss+beta*s_loss
    optimizer.zero_grad()
    tot_loss.backward()
    optimizer.step()

    if step %200 ==0:
      print(tot_loss)
      save_image(gen,r'./out/gen'+str(i)+'.png')


In [ ]:
!zip -r out.zip out